<table style="width: 100%;" id="nb-header>">
        <tr style="background-color: transparent;"><td>
            <img src="https://ds-connectors.github.io/econ-fa19/assets/images/blue_text.png" width="250px" style="margin-left: 0;" />
        </td><td>
            <p style="text-align: right; font-size: 10pt;"><strong>Economic Models</strong>, Spring 2020<br>
                Dr. Eric Van Dusen<br>
            Notebook by Andrei Caprau<br>
            Based on "Does the Stock Market Overreact?" by De Bondt and Thaler</p></td></tr>
    </table>

# Behavioral Economics Demonstration

In this demonstration we'll attempt to repeat the procedure De Bondt and Thaler use to show that people tend to "overreact" to sudden and dramatic news events, and that these overreactions are evident in stock prices.

This demonstration differs from De Bondt and Thaler's paper in that their data consisted of the prices of all stocks available to them from the Center for Research in Security Prices at the University of Chicago between 1926 and 1982. As such a large dataset of historical stock prices is difficult to obtain, we instead limit our data to a small selection of the largest public firms with listed prices between 2000 and 2019. As such, we will be cutting some corners and we will not produce statistically significant results as De Bondt and Thaler did. Furthermore, whatever results we do obtain we cannot extrapolate to the entire stock market as we have a potentially biased sample of stocks. We will be ignoring these problems entirely for now, and rather we will focus on the procedure itself.

## Procedure with one stock

Let's show how the data are cleaned and prepared for the study by first looking at only one stock. Let's load in data on AAPL.

In [ ]:
# We import these to allow us to read in files into the notebook.
import os, sys

# Numpy will allow us to work nicely with arrays. Pandas is a data 
# science package, much like the datascience package (confused yet?) 
# but better. Pandas is used in industry and in your future data 
# science classes.
import numpy as np
import pandas as pd

# This allows us to perform regular expressions. Don't worry about this.
import re

In [ ]:
aapl = pd.read_csv('data/AAPL.csv')
aapl.head()

Let's also load in data on dividends.

In [ ]:
aapl_div = pd.read_csv('dividends/AAPL.csv')
aapl_div.head()

Now let's combine these two datasets.

In [ ]:
aapl = aapl.join(aapl_div.set_index(keys='Date'), on='Date', how='left')
aapl.head()

In [ ]:
aapl['Dividends'] = aapl['Dividends'].fillna(value=0)
aapl['Adj Close'] = aapl['Adj Close'] + aapl['Dividends']
aapl = aapl.drop(columns=['Dividends'])
aapl.head()

We're only interested in monthly returns. To find this, we'll only look look at the closing price on the first day of each month. So let's do that. We create a column named 'First' which has the value 1 if the day in question is the first day of the month and 0 otherwise. We then reduce our dataset do just days where 'First' is 1.

In [ ]:
aapl['Year'] = aapl['Date'].str.split(r'-').str[0].astype(int)
aapl['Month'] = aapl['Date'].str.split(r'-').str[1].astype(int)
aapl['Day'] = aapl['Date'].str.split(r'-').str[2].astype(int)
aapl = aapl[['Year', 'Month', 'Day', 'Adj Close']]

aapl['PrevDay'] = aapl['Day'].shift().fillna(value=999).astype(int)
aapl['First'] = aapl.apply(lambda row: 1 if row['PrevDay'] > row['Day'] else 0, axis=1)

aapl.head()

In [ ]:
aapl = aapl[aapl['First'] == 1]
aapl.head()

Now we have to find the return for a particular month. To do this, we'll create a new column that is simply the column of closing price shifted down one. That way, each row not only has the closing price for that month, but also the month before it.

In [ ]:
aapl = aapl.drop(columns=['Day', 'PrevDay', 'First'])
aapl['Prev Close'] = aapl['Adj Close'].shift()
aapl.head()

Notice how the first row has no value for 'Prev Close'. Think very briefly about why that is. Let's drop that first row, and calculate return as a proportion.

In [ ]:
aapl = aapl.drop(index=0)
aapl['Return'] = aapl.apply(lambda row: (row['Adj Close'] - row['Prev Close']) 
                        / row['Prev Close'], axis=1)
aapl = aapl.reset_index(drop=True)
aapl.head()

Let's choose SPY to act as a proxy measurement for the performance of the market as a whole. Data on SPY is easy to obtain and has the same general form as AAPL data above. Let's do the same data manipulations on SPY as we did on AAPL.

In [ ]:
spy = pd.read_csv('data/SPY.csv')
spy_dividends = pd.read_csv('dividends/SPY.csv')
spy = spy.join(spy_dividends.set_index(keys='Date'), on='Date', how='left')
spy['Dividends'] = spy['Dividends'].fillna(value=0)
spy['Adj Close'] = spy['Adj Close'] + spy['Dividends']
spy = spy.drop(columns=['Dividends'])
spy['Year'] = spy['Date'].str.split(r'-').str[0].astype(int)
spy['Month'] = spy['Date'].str.split(r'-').str[1].astype(int)
spy['Day'] = spy['Date'].str.split(r'-').str[2].astype(int)
spy = spy[['Year', 'Month', 'Day', 'Adj Close']]
spy['PrevDay'] = spy['Day'].shift().fillna(value=999).astype(int)
spy['First'] = spy.apply(lambda row: 1 if row['PrevDay'] > row['Day'] else 0, axis=1)
spy = spy[spy['First'] == 1]
spy = spy.drop(columns=['Day', 'PrevDay', 'First'])
spy['Prev Close'] = spy['Adj Close'].shift()
spy = spy.drop(index=0)
spy['Return'] = spy.apply(lambda row: (row['Adj Close'] - row['Prev Close']) 
                        / row['Prev Close'], axis=1)
spy = spy.reset_index(drop=True)
spy.head()

Looks good. Now let's create a new column in AAPL that has the corresponding SPY return for each monthly return of AAPL.

In [ ]:
aapl['SPY Return'] = spy['Return']
aapl.head()

Now we can find excess return, which is the return of AAPL minus the return of SPY. It is a measure of how much better (or worse) AAPL's returns are compared to the market return.

In [ ]:
aapl['Excess Return'] = aapl['Return'] - aapl['SPY Return']
aapl.head()

We now have the monthly excess return for AAPL. Ideally we want to repeat this for every listed stock, but for now we only have a small subset of stocks to work with. As a result, our results are insignificant (and probably biased) and we are unable to come to De Bondt and Thaler's conclusions. However, we'll continue anyway so that we can get an idea and appreciation for what the process might feel like.

## Procedure with multiple stocks

First, let's repeat what we did above with AAPL but now with a larger subset of stocks.

In [ ]:
# All of our stock data, along with this notebook, are stored somewhere. 
# This gets that location.
cwd = os.getcwd()

# Here we read in the data for each stock that we have.
data = {}
for csv in os.listdir(cwd + '/data'):
    if csv == '.DS_Store':
        continue
    df = pd.read_csv('data/' + csv)
    data[re.findall(r'.+\.', str(csv))[0][:-1]] = df[['Date', 'Adj Close']]
    
# We also read in the data for dividends that were paid out for each 
# company.
dividends = {}
for csv in os.listdir(cwd + '/dividends'):
    if csv == '.DS_Store':
        continue
    dividends[re.findall(r'.+\.', str(csv))[0][:-1]] = pd.read_csv('dividends/' + csv)

# Here, we add dividend payouts to the closing prices of each stock, 
# since dividends must be included when calculating the return of a stock.
for stock in dividends.keys():
    df = data[stock]
    
    df = df.join(dividends[stock].set_index(keys='Date'), on='Date', how='left')
    
    df['Dividends'] = df['Dividends'].fillna(value=0)
    df['Adj Close'] = df['Adj Close'] + df['Dividends']
    df = df.drop(columns=['Dividends'])
    
    data[stock] = df

# Here we find out which days are the first days of the month, and we only 
# keep those days. Then, we create a new column that is the column 
# containing closing prices, but shifted down one row. This allows us 
# to compare the closing price in one month to the previous month, and 
# thus the return in that month.
for stock in data.keys():
    df = data[stock]
    
    df['Year'] = df['Date'].str.split(r'-').str[0].astype(int)
    df['Month'] = df['Date'].str.split(r'-').str[1].astype(int)
    df['Day'] = df['Date'].str.split(r'-').str[2].astype(int)
    df = df[['Year', 'Month', 'Day', 'Adj Close']]
    
    df['PrevDay'] = df['Day'].shift().fillna(value=999).astype(int)
    df['First'] = df.apply(lambda row: 1 if row['PrevDay'] > row['Day'] else 0, axis=1)
    df = df[df['First'] == 1]
    
    df = df.drop(columns=['Day', 'PrevDay', 'First'])
    df['Prev Close'] = df['Adj Close'].shift()
    df = df.drop(index=0)
    
    df['Return'] = df.apply(lambda row: (row['Adj Close'] - row['Prev Close']) 
                            / row['Prev Close'], axis=1)
    
    df = df.reset_index(drop=True)
    data[stock] = df

# We now add the return of $SPY, which will act as a proxy for market 
# return. We can then find excess return, which is a company's return 
# minus the return of the market.
SPY_return = data['SPY']['Return']
for stock in data.keys():
    if stock == 'SPY':
        continue
    df = data[stock]
    
    df['SPY Return'] = SPY_return
    df['Excess Return'] = df['Return'] - df['SPY Return']
    
    data[stock] = df

We form winning and losing portfolios based on cumulative returns during the last 3 years, and we observe the performance of these portfolios during the next 3 years. We repeat this for each non-overlapping 3-year block, starting from 2000 all the way to 2017. First, we decide that a winning and losing portfolio is just the best performing/worst performing stock. We then find the average cumulative excess return after portfolio formation for all 3-year periods, and take the difference between the losing portfolio and winning portfolio. We then compute a t-statistic for significance.

In [ ]:
# Create 3-year ranges during which we observe cumulative returns.
year_ranges = [[2000, 2001, 2002], [2003, 2004, 2005], 
               [2006, 2007, 2008], [2009, 2010, 2011], 
               [2012, 2013, 2014], [2015, 2016, 2017]]
winning_port = []
losing_port = []

# For each 3-year range, observe the cumulative returns in the previous 
# 3 years, for winning and losing portfolios, and then observe the 
# returns of these portfolios in the next 3 years.
for i in range(6):
    year_range = year_ranges[i]
    excess_returns = {}
    
    for stock in data.keys():
        if stock == 'SPY':
            continue
        df = data[stock]
        
        df = df[df.apply(lambda row: row['Year'] in year_range, axis=1)]
        cer = df['Excess Return'].sum()
        excess_returns[stock] = cer
        
    excess_returns = sorted(list(zip(excess_returns.keys(), excess_returns.values())), key=lambda a: a[1])
    losing_port.append(list(excess_returns[0]))
    winning_port.append(list(excess_returns[-1]))
    
    # Note: since we don't have complete 2020 data, we only observe 
    # the cumulative returns in 2018 and 2019 for portfolios made at 
    # the end of 2017.
    if i == 5:
        next_year_range = [2018, 2019]
    else:
        next_year_range = year_ranges[i + 1]
        
    df = data[losing_port[-1][0]]
    df = df[df.apply(lambda row: row['Year'] in next_year_range, axis=1)]
    losing_port[-1].append(df['Excess Return'].sum())
    
    df = data[winning_port[-1][0]]
    df = df[df.apply(lambda row: row['Year'] in next_year_range, axis=1)]
    winning_port[-1].append(df['Excess Return'].sum())

winning_port = np.array(winning_port)
losing_port = np.array(losing_port)

# Find the average cumulative returns of the winning and losing 
# portfolios, and compute a t-statistic to find significance.
acarw = np.mean(winning_port[:, 2].astype(float))
acarl = np.mean(losing_port[:, 2].astype(float))
s_sqrd = (np.sum(np.power(winning_port[:, 2].astype(float) - acarw, 2)) + 
          np.sum(np.power(losing_port[:, 2].astype(float) - acarl, 2))) / 2 * (6 - 1)

print("ACARl - ACARw: {}".format(acarl - acarw))
print("Pooled estimate of population variance: {}".format(s_sqrd))
print("t-statistic: {}".format((acarl - acarw) / np.sqrt(2 * s_sqrd / 6)))

This isn't very interesting. First, there is no significance, and second, it seems that the difference leans a bit on the negative side. Why might these two things be the case?

Below, we now decide that a winning portfolio is the top 5 best performing stocks, and a losing portfolio is the bottom 5 worst performing stocks. Let's repeat.

In [ ]:
year_ranges = [[2000, 2001, 2002], [2003, 2004, 2005], 
               [2006, 2007, 2008], [2009, 2010, 2011], 
               [2012, 2013, 2014], [2015, 2016, 2017]]
winning_port = []
losing_port = []
winning_car = []
losing_car = []

for i in range(6):
    year_range = year_ranges[i]
    excess_returns = {}
    
    for stock in data.keys():
        if stock == 'SPY':
            continue
        df = data[stock]
        
        df = df[df.apply(lambda row: row['Year'] in year_range, axis=1)]
        cer = df['Excess Return'].sum()
        excess_returns[stock] = cer
        
    excess_returns = sorted(list(zip(excess_returns.keys(), excess_returns.values())), key=lambda a: a[1])
    losing_port.append(np.array(excess_returns[0:5]))
    winning_port.append(np.array(excess_returns[::-1][0:5]))
    
    if i == 5:
        next_year_range = [2018, 2019]
    else:
        next_year_range = year_ranges[i + 1]
        
    carl = 0
    for stock in losing_port[-1][:, 0]:
        df = data[stock]
        df = df[df.apply(lambda row: row['Year'] in next_year_range, axis=1)]
        carl += df['Excess Return'].sum()
    losing_car.append(carl / len(losing_port[-1][:, 0]))
    
    carw = 0
    for stock in winning_port[-1][:, 0]:
        df = data[stock]
        df = df[df.apply(lambda row: row['Year'] in next_year_range, axis=1)]
        carw += df['Excess Return'].sum()
    winning_car.append(carw / len(winning_port[-1][:, 0]))

acarw = np.mean(winning_car)
acarl = np.mean(losing_car)
s_sqrd = (np.sum(np.power(winning_car - acarw, 2)) + 
          np.sum(np.power(losing_car - acarl, 2))) / 2 * (6 - 1)
(acarl - acarw) / np.sqrt(2 * s_sqrd / 6)

print("ACARl - ACARw: {}".format(acarl - acarw))
print("Pooled estimate of population variance: {}".format(s_sqrd))
print("t-statistic: {}".format((acarl - acarw) / np.sqrt(2 * s_sqrd / 6)))

We see that our results are still insignificant, but the difference in average cumulative returns between the losing and winning portfolios maybe started leaning more towards the positive side. Why might this change have come about?

## Conclusions

As we foreshadowed earlier, De Bondt and Thaler showed with strong confidence that people overreact to news, whereas our results are not conclusive in any way. What changes should we make in our study to try to have solid evidence in favor of the hypothesis that people overreact to news?